# --- Day 5: If You Give A Seed A Fertilizer ---

https://adventofcode.com/2023/day/5

Not gonna use/need this, turns out, but this is a pretty neat trick none-the-less:

In [3]:
class IdentityDict(dict):
    __missing__ = lambda self, key : key

test = IdentityDict()
test[1]   

1

## Parse the Input Data

In [4]:
import re
from collections import defaultdict

In [5]:
def parse_input(filename):
    """Parse input data for puzzle.

    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.

    Returns
    -------
    seeds, maps : list, dict
        seeds is a list of the seeds that need to be run through all the mappings;
        maps is a dictionary of lists, each list containing three numbers:
            * start of destination range
            * start of source range
            * range length
    """
    maps = defaultdict(list)
    map_keys = ["seed2soil", "soil2fert", "fert2water", "water2light", "light2temp", "temp2hum", "hum2loc"]
    
    with open(f'../inputs/{filename}.txt') as _file:
        for i, line in enumerate(_file):
            if i == 0:
                seeds = [int(s) for s in re.findall("\d+", line)]

            else:
                if line == "\n":
                    map_key = map_keys.pop(0)
                else:
                    if line[0].isdigit():
                        maps[map_key].append([int(x) for x in line.strip().split()])

    return seeds, maps

In [6]:
parse_input("test_almanac")

([79, 14, 55, 13],
 defaultdict(list,
             {'seed2soil': [[50, 98, 2], [52, 50, 48]],
              'soil2fert': [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
              'fert2water': [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
              'water2light': [[88, 18, 7], [18, 25, 70]],
              'light2temp': [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
              'temp2hum': [[0, 69, 1], [1, 0, 69]],
              'hum2loc': [[60, 56, 37], [56, 93, 4]]}))

## Part 1
---

In [7]:
def solve1(seeds, maps):
    min_loc = float("inf")
    map_keys = ["seed2soil", "soil2fert", "fert2water", "water2light", "light2temp", "temp2hum", "hum2loc"]

    for seed in seeds:
        for key in map_keys:
            for line in maps[key]:
                d, s, r = line  # d = destination, s = source, r = range
                if seed in range(s, s+r):
                    seed = d + (seed - s)  # no need to change the seed name...
                    break
                # ...if seed isn't found in any of the map ranges, it remains the same value

        min_loc = min(min_loc, seed)

    return min_loc

### Run on Test Data

In [8]:
solve1(*parse_input("test_almanac")) == 35

True

### Run on Input Data

In [9]:
solve1(*parse_input("almanac"))

403695602

## Part 2
---

In [10]:
from collections import deque

In [50]:
def solve2(seeds, maps, debug=False):
    starts = []
    map_keys = ["seed2soil", "soil2fert", "fert2water", "water2light", "light2temp", "temp2hum", "hum2loc"]

    starts = [x for i, x in enumerate(seeds) if i % 2 == 0]
    ranges = [x for i, x in enumerate(seeds) if i % 2 == 1] 

    seed_ranges = deque([[s, s + r - 1] for s, r in zip(starts, ranges)])

    print(seed_ranges)

    while seed_ranges:
        seed_start, seed_end = seed_ranges.popleft()
        for key in map_keys:
            if debug: print(key)
            for map_range in maps[key]:
                d, s, r = map_range  # d = destination, s = source, r = range
                map_start, map_end = s, s + r - 1
                offset = d - s
                print(d, s, offset)
                
                # Seed range to the left or right of current map range
                if seed_end < map_start or map_end < seed_start:
                    continue

                # Seed range completely inside map range
                elif map_start <= seed_start and seed_end <= map_end:
                    if debug: print("inside:", f"seed range: {[seed_start, seed_end]}, map range: {[map_start, map_end]}")
                    seed_start += offset 
                    seed_end += offset 
                    if debug: print(f"inside: offset: {offset}, new seed range: {[seed_start, seed_end]}")
                    break

                # Seed range left overlap map range
                elif map_start <= seed_end:
                    if debug: print(f"no overlap: seed range: {[seed_start, seed_end]}, map range: {[map_start, map_end]}")
                    seed_ranges.appendleft([seed_start, map_start - 1])
                    seed_start = map_start + offset
                    seed_end = seed_end + offset
                    if debug: print(f"no overlap: offset: {offset}, new seed range: {[seed_start, seed_end]}")
                    break

                # Seed range right overlap map range
                elif seed_start <= map_end:
                    if debug: print(f"right overlap: seed: {[seed_start, seed_end]}, map: {[map_start, map_end]}")
                    seed_ranges.appendleft([map_end + 1, seed_end])
                    seed_start = seed_start + offset
                    seed_end = map_end + offset
                    break

                else:
                    if debug: print("Ack!")
        

        starts.append(seed_start)

    return min(starts)

### Run on Test Data

In [51]:
solve2(*parse_input("test_almanac"), debug=True) #== 46

deque([[79, 92], [55, 67]])
seed2soil
50 98 -48
52 50 2
inside: seed range: [79, 92], map range: [50, 97]
inside: offset: 2, new seed range: [81, 94]
soil2fert
0 15 -15
37 52 -15
39 0 39
fert2water
49 53 -4
0 11 -11
42 0 42
57 7 50
water2light
88 18 70
18 25 -7
inside: seed range: [81, 94], map range: [25, 94]
inside: offset: -7, new seed range: [74, 87]
light2temp
45 77 -32
no overlap: seed range: [74, 87], map range: [77, 99]
no overlap: offset: -32, new seed range: [45, 55]
temp2hum
0 69 -69
1 0 1
inside: seed range: [45, 55], map range: [0, 68]
inside: offset: 1, new seed range: [46, 56]
hum2loc
60 56 4
no overlap: seed range: [46, 56], map range: [56, 92]
no overlap: offset: 4, new seed range: [60, 60]
seed2soil
50 98 -48
52 50 2
no overlap: seed range: [46, 55], map range: [50, 97]
no overlap: offset: 2, new seed range: [52, 57]
soil2fert
0 15 -15
37 52 -15
no overlap: seed range: [52, 57], map range: [52, 53]
no overlap: offset: -15, new seed range: [37, 42]
fert2water
49 53 -4


20

### Run on Input Data

In [34]:
solve2(*parse_input("almanac"))

inf